In [1]:
#https://huggingface.co/docs/transformers/model_doc/t5
#!pip install t5

In [2]:
#!pip install pyterrier

In [3]:
#!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.

# Import Libraries

In [4]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



# Connect To Google Drive And Load Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls '/content/drive/MyDrive/project_data'

scifact


In [7]:
!ls '/content/drive/MyDrive/project_data/scifact'

corpus.jsonl	    queries.jsonl  test.source_rl  train.source     train.target_rl  val.target
process_scifact.py  test.csv	   test.target	   train.source_rl  val.source	     val.target_rl
qrels		    test.source    train.csv	   train.target     val.source_rl


In [8]:
data_directory = '/content/drive/MyDrive/project_data'

# Load Meta-Data

In [9]:
corpus = pd.read_json(data_directory + '/scifact/corpus.jsonl', lines=True, dtype=str)

corpus

,_id,title,text,metadata
0,4983,Microstructural development of human newborn c...,Alterations of the architecture of cerebral wh...,{}
1,5836,Induction of myelodysplasia by myeloid-derived...,Myelodysplastic syndromes (MDS) are age-depend...,{}
2,7912,"BC1 RNA, the transcript from a master gene for...",ID elements are short interspersed elements (S...,{}
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,DNA methylation plays an important role in bio...,{}
4,19238,The human myelin basic protein gene is include...,Two human Golli (for gene expressed in the oli...,{}
...,...,...,...,...
5178,195689316,Body-mass index and cause-specific mortality i...,BACKGROUND The main associations of body-mass ...,{}
5179,195689757,Targeting metabolic remodeling in glioblastoma...,A key aberrant biological difference between t...,{}
5180,196664003,Signaling architectures that transmit unidirec...,A signaling pathway transmits information from...,{}
5181,198133135,"Association between pre-diabetes, type 2 diabe...",AIMS Trabecular bone score (TBS) is a surrogat...,{}


In [10]:
queries = pd.read_json(data_directory + '/scifact/queries.jsonl', lines=True, dtype=str)
queries

,_id,text,metadata
0,0,0-dimensional biomaterials lack inductive prop...,{}
1,2,1 in 5 million in UK have abnormal PrP positiv...,"{'13734012': [{'sentences': [4], 'label': 'CON..."
2,4,1-1% of colorectal cancer patients are diagnos...,{}
3,6,10% of sudden infant death syndrome (SIDS) dea...,{}
4,9,32% of liver transplantation programs required...,"{'44265107': [{'sentences': [15], 'label': 'SU..."
...,...,...,...
1104,1379,Women with a higher birth weight are more like...,"{'16322674': [{'sentences': [5], 'label': 'SUP..."
1105,1382,aPKCz causes tumour enhancement by affecting g...,"{'17755060': [{'sentences': [3], 'label': 'CON..."
1106,1385,cSMAC formation enhances weak ligand signalling.,"{'306006': [{'sentences': [4], 'label': 'SUPPO..."
1107,1389,mTORC2 regulates intracellular cysteine levels...,"{'23895668': [{'sentences': [2, 3], 'label': '..."


# Load Train and Test data and qrels from .csv files

In [11]:
#train_data
df_train = pd.read_csv(data_directory + '/scifact/train.csv', sep='\t', dtype=str)

print(df_train.shape)
print(df_train['query'].apply(len).mean())
print(df_train['text'].apply(len).mean())
df_train

(919, 3)
88.82372143634385
1535.5070729053318


,qid,query,text
0,0,0-dimensional biomaterials lack inductive prop...,Nanotechnologies are emerging platforms that c...
1,2,1 in 5 million in UK have abnormal PrP positiv...,OBJECTIVES To carry out a further survey of ar...
2,4,1-1% of colorectal cancer patients are diagnos...,CONTEXT Medicare's reimbursement policy was ch...
3,6,10% of sudden infant death syndrome (SIDS) dea...,Despite declines in prevalence during the past...
4,9,32% of liver transplantation programs required...,ContextChronic hepatitis C is the leading caus...
...,...,...,...
914,1404,siRNA knockdown of A20 slows tumor progression...,OBJECTIVE A20 is a TNF-inducible primary respo...
915,1405,taa1 tar1 tar2 triple mutants in Arabidopsis d...,Plants have evolved a tremendous ability to re...
916,1406,β-sheet opening occurs during pleurotolysin po...,Membrane attack complex/perforin-like (MACPF) ...
917,1407,β1/Ketel is able to bind microtubules.,The microtubule (MT) cytoskeleton is required ...


In [12]:
#train qrels
df_train_qrels = pd.read_csv(data_directory + '/scifact/qrels/train.tsv', sep='\t', dtype=str)

df_train_qrels

,query-id,corpus-id,score
0,0,31715818,1
1,2,13734012,1
2,4,22942787,1
3,6,2613775,1
4,9,44265107,1
...,...,...,...
914,1404,38355793,1
915,1405,10504681,1
916,1406,2617858,1
917,1407,8087082,1


In [13]:
#test data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
print(df_test.shape)
print(df_test['query'].apply(len).mean())
print(df_test['text'].apply(len).mean())
df_test

(339, 3)
91.43952802359883
1520.1592920353983


,qid,query,text
0,1,0-dimensional biomaterials show inductive prop...,Nanotechnologies are emerging platforms that c...
1,3,"1,000 genomes project enables mapping of genet...",Genome-wide association studies (GWAS) have no...
2,5,1/2000 in UK have abnormal PrP positivity.,OBJECTIVES To carry out a further survey of ar...
3,13,5% of perinatal mortality is due to low birth ...,CONTEXT One key target of the United Nations M...
4,36,A deficiency of vitamin B12 increases blood le...,BACKGROUND Homocysteine is a risk factor for c...
...,...,...,...
334,1379,Women with a higher birth weight are more like...,"INTRODUCTION Various perinatal factors, includ..."
335,1382,aPKCz causes tumour enhancement by affecting g...,Tumor cells have high-energetic and anabolic n...
336,1385,cSMAC formation enhances weak ligand signalling.,T cell activation is predicated on the interac...
337,1389,mTORC2 regulates intracellular cysteine levels...,Mutations in cancer reprogram amino acid metab...


In [14]:
#test_data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
df_test2 = df_test[['qid', 'query']]
df_test2.to_csv('my_test_queries.csv', sep = '\t', index=False, header=False)
test_query = pt.io.read_topics('my_test_queries.csv', format='singleline')
test_query

,qid,query
0,1,0 dimensional biomaterials show inductive prop...
1,3,1 000 genomes project enables mapping of genet...
2,5,1 2000 in uk have abnormal prp positivity
3,13,5 of perinatal mortality is due to low birth w...
4,36,a deficiency of vitamin b12 increases blood le...
...,...,...
334,1379,women with a higher birth weight are more like...
335,1382,apkcz causes tumour enhancement by affecting g...
336,1385,csmac formation enhances weak ligand signalling
337,1389,mtorc2 regulates intracellular cysteine levels...


In [15]:
#test qrels
df_test_qrels = pd.read_csv(data_directory + '/scifact/qrels/test.tsv', sep='\t', dtype=str)

df_test_qrels

,query-id,corpus-id,score
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


In [16]:
#source file
train_source = df_train['query']
test_source = df_test['query']
test_source

0      0-dimensional biomaterials show inductive prop...
1      1,000 genomes project enables mapping of genet...
2             1/2000 in UK have abnormal PrP positivity.
3      5% of perinatal mortality is due to low birth ...
4      A deficiency of vitamin B12 increases blood le...
                             ...                        
334    Women with a higher birth weight are more like...
335    aPKCz causes tumour enhancement by affecting g...
336     cSMAC formation enhances weak ligand signalling.
337    mTORC2 regulates intracellular cysteine levels...
338    p16INK4A accumulation is  linked to an abnorma...
Name: query, Length: 339, dtype: object

In [17]:
train_target = df_train['text']
test_target = df_test['text']

test_target

0      Nanotechnologies are emerging platforms that c...
1      Genome-wide association studies (GWAS) have no...
2      OBJECTIVES To carry out a further survey of ar...
3      CONTEXT One key target of the United Nations M...
4      BACKGROUND Homocysteine is a risk factor for c...
                             ...                        
334    INTRODUCTION Various perinatal factors, includ...
335    Tumor cells have high-energetic and anabolic n...
336    T cell activation is predicated on the interac...
337    Mutations in cancer reprogram amino acid metab...
338    Monitoring cancer and aging in vivo remains ex...
Name: text, Length: 339, dtype: object

# T5 Implementation for Query Rewriting

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Training T5 model

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# the following 2 hyperparameters are task-specific
max_source_length = 64
max_target_length = 256
batch_size = 16


input_sequences = []
output_sequences = []
for i in range(len(train_source)):
  input_sequences.append(train_source.iloc[i])
  output_sequences.append(train_target.iloc[i])
  if i % batch_size == 0:
        encoding = tokenizer(
          [sequence for sequence in input_sequences],
          padding="longest",
          max_length=max_source_length,
          truncation=True,
          return_tensors="pt")
        input_ids = encoding.input_ids.to(device)
        attention_mask = encoding.attention_mask.to(device)

        # encode the targets
        target_encoding = tokenizer(
            output_sequences,
            padding="longest",
            max_length=max_target_length,
            truncation=True,
            return_tensors="pt",
        )
        labels = target_encoding.input_ids.to(device)
        # replace padding token id's of the labels by -100 so it's ignored by the loss
        labels[labels == tokenizer.pad_token_id] = -100

        # forward pass
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
        print(loss.item())
        input_sequences = []
        output_sequences = []

6.640742301940918
14.473292350769043


# Generating Alternative Queries

In [ ]:
input_sequences2 = []
output_sequences2 = []
inputs = []
outputs = []
rewritten_texts = []
for i in range(len(test_query)):
  #input_sequences2.append(test_source.iloc[i])
  input_sequences2.append(test_query['query'].iloc[i])
  output_sequences2.append(test_target.iloc[i])
  encoding = tokenizer(
          #test_source.iloc[i],
          test_query['query'].iloc[i],
          padding="longest",
          max_length=max_source_length,
          truncation=True,
          return_tensors="pt")
  input_ids = encoding.input_ids.to(device)
  attention_mask = encoding.attention_mask.to(device)
  inputs.append(input_ids)
  output = model.generate(input_ids)
  outputs.append(output)
  rewritten_texts.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
test_query['rewritten_texts'] = rewritten_texts
test_query['combi'] = test_query['query'] + ' ' + rewritten_texts
test_query

# Evaluation

In [ ]:
corpus = corpus.rename(columns={'_id': 'docno'})
corpus

In [ ]:
indexer = pt.DFIndexer("./indexes_scifact/both", overwrite=True, blocks=True,verbose=True, stemmer='porter', stopwords='terrier', tokenizer = 'english')
index_ref = indexer.index(corpus["text"], corpus["docno"])
index = pt.IndexFactory.of(index_ref)

In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
new_queries = pd.DataFrame()
new_queries['qid'] = test_query['qid']
new_queries['query'] = rewritten_texts

In [ ]:
new_queries2 = pd.DataFrame()
new_queries2['qid'] = test_query['qid']
new_queries2['query'] = test_query['combi']

In [ ]:
test_qrels = df_test_qrels.rename(columns={"query-id": "qid", "corpus-id" : "docno", "score": "label"})
test_qrels['iteration'] = 0
qrels = test_qrels.astype({'label': 'int32'})
qrels

In [ ]:
final_res_ndcg = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg)
print(final_res_map)
print(final_res_precision)
print(final_res_recall)

In [ ]:
final_res_ndcg2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall2 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries2,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg2)
print(final_res_map2)
print(final_res_precision2)
print(final_res_recall2)

# Rewrite Queries - google/flan-t5-base

In [ ]:
#https://codepal.ai/code-generator/query/DpMCp564/ai-content-rewriter
model2 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small").to(device)
tokenizer2 = T5Tokenizer.from_pretrained("google/flan-t5-small")

In [ ]:
max_source_length = 64
max_target_length = 128
batch_size = 8

input_sequences2 = []
output_sequences2 = []
for i in range(len(train_source)):
  input_sequences2.append(train_source.iloc[i])
  output_sequences2.append(train_target.iloc[i])
  if i % batch_size == 0:
        encoding = tokenizer2(
          [sequence for sequence in input_sequences2],
          padding="longest",
          max_length=max_source_length,
          truncation=True,
          return_tensors="pt")
        input_ids = encoding.input_ids.to(device)
        attention_mask = encoding.attention_mask.to(device)

        # encode the targets
        target_encoding = tokenizer2(
            output_sequences2,
            padding="longest",
            max_length=max_target_length,
            truncation=True,
            return_tensors="pt",
        )
        labels = target_encoding.input_ids.to(device)
        # replace padding token id's of the labels by -100 so it's ignored by the loss
        labels[labels == tokenizer2.pad_token_id] = -100

        # forward pass
        loss = model2(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
        print(loss.item())
        input_sequences2 = []
        output_sequences2 = []

In [ ]:
rewritten_texts2 = []
for i in range(len(test_query)):
  # Tokenize the input text
  inputs = tokenizer2.encode(test_query['query'].iloc[i], return_tensors="pt").to(device)

  # Generate the rewritten text
  outputs = model2.generate(inputs, max_length=100, early_stopping=True)
  rewritten_text2 = tokenizer2.decode(outputs[0], skip_special_tokens=True)
  rewritten_texts2.append(rewritten_text2)

In [ ]:
new_queries3 = pd.DataFrame()
new_queries3['qid'] = test_query['qid']
new_queries3['query'] = rewritten_texts3

# Evaluation

In [ ]:
final_res_ndcg3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["ndcg_cut_10", "ndcg_cut_100", "ndcg_cut_1000"])

final_res_map3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["map_cut_10", "map_cut_100", "map_cut_1000"])

final_res_precision3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["P_10", "P_100", "P_1000"])

final_res_recall3 = pt.Experiment(
                        retr_systems = [bm25],
                        names =  ["BM25"],
                        topics = new_queries3,
                        qrels = qrels,
                        eval_metrics = ["recall_10", "recall_100", "recall_1000"])

print(final_res_ndcg3)
print(final_res_map3)
print(final_res_precision3)
print(final_res_recall3)

In [ ]:
#Different approaches to make LLM trainable:

#1. Train LLM by having some input which is query and output which part of a document
#1. From here during training try to fine-tune LLMs hyper-parameters.
#1. https://huggingface.co/docs/transformers/model_doc/t5
#1. Use supervision learning from above link to train T5 LLM.
#1. Compare untrained T5 with Trained T5.
#2. Use Markov Decision Problem to compute reward and train T5 that way. (Alternative approach to 1)